### Importing libraries

In [5]:
import os
os.chdir(r'C:\Users\tejan\Downloads')
import functools
import pandas as pd


### Reading in the opioid prescription rates for Medicare Part D frm 2014-2017

In [6]:
df_17 = pd.read_csv('2017.csv')
df_16 = pd.read_csv('2016.csv')
df_15 = pd.read_csv('2015.csv')
df_14 = pd.read_csv('2014.csv')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (36,53,55,58,60,62,64,69,70,71,72,73,74,76,77,78,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (36,53,58,62,64,69,70,71,72,73,74,76,77,78,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merge all the datasets on NPI to get the final dataset

In [47]:
final_df = functools.reduce(functools.partial(pd.merge, on='npi'), [
                        df_17[['npi','opioid_prescriber_rate','nppes_provider_state','specialty_description']].rename(columns = {'opioid_prescriber_rate' :'op_17'}),
                        df_16[['npi','opioid_prescriber_rate']].rename(columns = {'opioid_prescriber_rate' :'op_16'}), 
                        df_15[['npi','opioid_prescriber_rate']].rename(columns = {'opioid_prescriber_rate' :'op_15'}),
                        df_14[['npi','opioid_prescriber_rate']].rename(columns = {'opioid_prescriber_rate' :'op_14'})])


In [48]:
final_df.head()

,npi,op_17,nppes_provider_state,specialty_description,op_16,op_15,op_14
0,1003000126,3.69,MD,Internal Medicine,4.22,4.61,5.48
1,1003000142,53.44,OH,Anesthesiology,54.30,48.21,45.36
2,1003000167,25.45,NV,Dentist,22.45,37.21,31.03
3,1003000282,0.00,TN,Nurse Practitioner,NaN,18.42,0.00
4,1003000407,1.15,PA,Family Practice,0.76,1.87,4.76


In [50]:
final_df.dropna(axis=0,inplace=True)

In [51]:
final_df.set_index('npi')

,op_17,nppes_provider_state,specialty_description,op_16,op_15,op_14
npi,,,,,,
1003000126,3.69,MD,Internal Medicine,4.22,4.61,5.48
1003000142,53.44,OH,Anesthesiology,54.30,48.21,45.36
1003000167,25.45,NV,Dentist,22.45,37.21,31.03
1003000407,1.15,PA,Family Practice,0.76,1.87,4.76
1003000522,2.57,FL,Family Practice,3.53,3.60,3.62
1003000530,2.34,PA,Internal Medicine,3.65,3.86,2.96
1003000597,11.93,OK,Urology,12.77,18.63,29.87
1003000639,0.00,CA,Cardiac Surgery,0.00,0.00,0.00
1003000902,2.13,KY,Family Practice,2.34,3.44,2.63


### Find the median prescription rates aggregated by provider's state and specialty of the provider

In [56]:
median_17=final_df.groupby(['nppes_provider_state','specialty_description'])['op_17'].median().reset_index().rename(columns={'op_17':'medianop_17'})
median_16=final_df.groupby(['nppes_provider_state','specialty_description'])['op_16'].median().reset_index().rename(columns={'op_16':'medianop_16'})
median_15=final_df.groupby(['nppes_provider_state','specialty_description'])['op_15'].median().reset_index().rename(columns={'op_15':'medianop_15'})
median_14=final_df.groupby(['nppes_provider_state','specialty_description'])['op_14'].median().reset_index().rename(columns={'op_14':'medianop_14'})

### Adding the median values to the final dataset on state, specialty level

In [59]:
median_df = functools.reduce(functools.partial(pd.merge, on=['nppes_provider_state','specialty_description']),
                            [median_17,median_16,median_15,median_14])

In [60]:
median_df.head()

,nppes_provider_state,specialty_description,medianop_17,medianop_16,medianop_15,medianop_14
0,AA,Dentist,15.490,18.29,19.05,26.470
1,AA,Diagnostic Radiology,0.000,0.00,0.00,0.000
2,AA,Family Practice,0.000,5.57,15.96,13.640
3,AA,Oral Surgery (Dentist only),30.140,28.46,28.10,30.460
4,AE,Dentist,27.015,26.11,32.93,31.805


In [71]:
data_calc=pd.merge(final_df,median_df,how='left',on=['nppes_provider_state','specialty_description'])
data_calc.head()

,npi,op_17,nppes_provider_state,specialty_description,op_16,op_15,op_14,medianop_17,medianop_16,medianop_15,medianop_14
0,1003000126,3.69,MD,Internal Medicine,4.22,4.61,5.48,2.80,3.24,3.60,4.05
1,1003000142,53.44,OH,Anesthesiology,54.30,48.21,45.36,46.38,46.66,46.41,47.35
2,1003000167,25.45,NV,Dentist,22.45,37.21,31.03,23.44,23.61,24.89,24.36
3,1003000407,1.15,PA,Family Practice,0.76,1.87,4.76,3.15,3.63,3.84,4.04
4,1003000522,2.57,FL,Family Practice,3.53,3.60,3.62,3.23,3.50,3.68,3.99


### Creating labels if the prescription rate for the provider is above the median value

In [72]:
data_calc['label_17']=0
data_calc['label_16']=0
data_calc['label_15']=0
data_calc['label_14']=0
data_calc['label_17'][data_calc['op_17']>=data_calc['medianop_17']]=1
data_calc['label_16'][data_calc['op_16']>=data_calc['medianop_16']]=1
data_calc['label_15'][data_calc['op_15']>=data_calc['medianop_15']]=1
data_calc['label_14'][data_calc['op_14']>=data_calc['medianop_14']]=1

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [73]:
data_calc.head()

,npi,op_17,nppes_provider_state,specialty_description,op_16,op_15,op_14,medianop_17,medianop_16,medianop_15,medianop_14,label_17,label_16,label_15,label_14
0,1003000126,3.69,MD,Internal Medicine,4.22,4.61,5.48,2.80,3.24,3.60,4.05,1,1,1,1
1,1003000142,53.44,OH,Anesthesiology,54.30,48.21,45.36,46.38,46.66,46.41,47.35,1,1,1,0
2,1003000167,25.45,NV,Dentist,22.45,37.21,31.03,23.44,23.61,24.89,24.36,1,0,1,1
3,1003000407,1.15,PA,Family Practice,0.76,1.87,4.76,3.15,3.63,3.84,4.04,0,0,0,1
4,1003000522,2.57,FL,Family Practice,3.53,3.60,3.62,3.23,3.50,3.68,3.99,0,1,0,0


### Finding how many times a provider has been a repetitive offender in terms of exceeding the median prescription rate for satte and specialty

In [74]:
data_calc['total_flags']=0
data_calc['total_flags'] = data_calc['label_17']+data_calc['label_16']+data_calc['label_15']+data_calc['label_14']

In [75]:
data_calc.head()

,npi,op_17,nppes_provider_state,specialty_description,op_16,op_15,op_14,medianop_17,medianop_16,medianop_15,medianop_14,label_17,label_16,label_15,label_14,total_flags
0,1003000126,3.69,MD,Internal Medicine,4.22,4.61,5.48,2.80,3.24,3.60,4.05,1,1,1,1,4
1,1003000142,53.44,OH,Anesthesiology,54.30,48.21,45.36,46.38,46.66,46.41,47.35,1,1,1,0,3
2,1003000167,25.45,NV,Dentist,22.45,37.21,31.03,23.44,23.61,24.89,24.36,1,0,1,1,3
3,1003000407,1.15,PA,Family Practice,0.76,1.87,4.76,3.15,3.63,3.84,4.04,0,0,0,1,1
4,1003000522,2.57,FL,Family Practice,3.53,3.60,3.62,3.23,3.50,3.68,3.99,0,1,0,0,1


In [76]:
data_calc['total_flags'].value_counts(normalize=True)

4    0.460538
0    0.296592
3    0.089948
1    0.077808
2    0.075114
Name: total_flags, dtype: float64

In [80]:
data_calc.groupby(['nppes_provider_state','specialty_description'])['total_flags'].agg(lambda x:x.value_counts().index[0]).reset_index()

,nppes_provider_state,specialty_description,total_flags
0,AA,Dentist,4
1,AA,Diagnostic Radiology,4
2,AA,Family Practice,4
3,AA,Oral Surgery (Dentist only),4
4,AE,Dentist,3
5,AE,Emergency Medicine,4
6,AE,Family Practice,4
7,AE,Nurse Practitioner,4
8,AE,Optometry,4
9,AE,Physician Assistant,4


### Anova test to check if the mean value of average prescription rate has varied across all the years for each state, specialty type

In [39]:
import scipy.stats as stats

In [79]:
stats.f_oneway(final_df['op_14'], final_df['op_15'], final_df['op_16'], final_df['op_17'])

F_onewayResult(statistic=856.0763550776948, pvalue=0.0)